#### 1. 누락 경로 필터링 (LibriSpeech/train-other-500)

#### 2. stage-1
- task별로 분리해준 후에 
- 오디오 길이 기준 이상치(상-하위 5%)를 제거하고
- 다시 텍스트 기준 10분위 분할한 후에
- 각 분위에서 설정한 갯수에 맞춰 샘플링

#### 3. stage-2
- task별로 분리해준 후에 
- 오디오 길이 기준 이상치(상-하위 5%)를 제거하고
- 다시 텍스트 기준 10분위 분할한 후에
- 각 분위에서 설정한 갯수에 맞춰 샘플링
- filtered_stage1_train.json에 대하여 asr, audiocaption task 각각에 대해
오디오 길이 기준 상위 하위 5퍼센트를 제외하고, 중복을 피해 각 1만개씩만 stage-2로 추가 샘플링

#### 4. 중복경로 제거


In [ ]:
# 1. 누락경로 제거

import json
import os

def filter_librispeech_other500(input_files):
    for file_path in input_files:
        # 파일 읽기
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # 처리 전 개수 저장
        before_count = len(data['annotation'])
        
        # LibriSpeech/train-other-500으로 시작하는 path를 가진 항목 필터링
        filtered_annotations = [
            item for item in data['annotation'] 
            if not item['path'].startswith('/LibriSpeech/train-other-500')
        ]
        
        # 필터링된 결과로 데이터 업데이트
        data['annotation'] = filtered_annotations
        
        # 처리 후 개수 계산
        after_count = len(data['annotation'])
        
        # 결과 출력
        print(f"File: {file_path}")
        print(f"Before filtering: {before_count} items")
        print(f"After filtering: {after_count} items")
        print(f"Removed: {before_count - after_count} items\n")
        
        # 현재 폴더에 새 파일명으로 저장
        original_filename = os.path.basename(file_path)
        new_filename = f"filtered_{original_filename}"
        
        # 한 줄로 저장 (공백 없이)
        with open(new_filename, 'w', encoding='utf-8') as f:
            f.write(json.dumps(data, ensure_ascii=False, separators=(', ', ': ')))
        
        print(f"Saved as: {new_filename}\n")

# 파일 경로 리스트
input_files = ['stage1_train.json', 'stage2_train.json']

# 실행
filter_librispeech_other500(input_files)